In [ ]:
from __future__ import print_function, division

from torchvision import datasets, models, transforms
from imgaug import augmenters as iaa
from sklearn.externals import joblib
from torch.optim import lr_scheduler
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn as nn
from PIL import Image
import imgaug as ia
import pandas as pd
import numpy as np
import torchvision
import pickle
import joblib
import torch
import copy
import time
import os

In [1]:
class ImgAugTransform:
  def __init__(self):
    self.aug = iaa.Sequential([
        iaa.Scale((224, 224)),
        iaa.Sometimes(0.25, iaa.GaussianBlur(sigma=(0, 3.0))),
        iaa.Fliplr(0.5),
        iaa.Affine(rotate=(-20, 20), mode='symmetric'),
        iaa.Sometimes(0.25,
                      iaa.OneOf([iaa.Dropout(p=(0, 0.1)),
                                 iaa.CoarseDropout(0.1, size_percent=0.5)])),
        iaa.AddToHueAndSaturation(value=(-10, 10), per_channel=True)
    ])
      
  def __call__(self, img):
    img = np.array(img)
    return self.aug.augment_image(img)


data_transforms = transforms.Compose([
        ImgAugTransform(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

model_ft = joblib.load("../GoogleStatic_ResNet18.sav")

data = pd.read_csv("./y1314_df.csv")

df = pd.DataFrame()
cfail = []
cpass = []
ids = []
label = []

directory = "./jpg/"

for filename in os.listdir(directory):
    school_id = filename[0:6]
    ids.append(school_id)
    info = data[data['school_id'] == int(school_id)]
    label.append(info.intervention.tolist()[0])
    img = Image.open(directory + filename)
    cnn_input = data_transforms(img).unsqueeze(0)
    model_ft.eval()
    pred = model_ft(cnn_input)
    print(pred) 
    preds = pred.softmax(1).tolist()
    cfail.append(preds[0][0])
    cpass.append(preds[0][1])
    

/usr/local/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
/usr/local/lib/python3.7/site-packages/imgaug/imgaug.py:182: DeprecationWarning: Function `Scale()` is deprecated. Use `Resize` instead. Resize has the exactly same interface as Scale.
  warn_deprecated(msg, stacklevel=3)


tensor([[0.0881, 1.6036]], grad_fn=<AddmmBackward>)
tensor([[ 1.5022, -0.2157]], grad_fn=<AddmmBackward>)
tensor([[ 1.1542, -0.8852]], grad_fn=<AddmmBackward>)
tensor([[-1.6195,  4.2671]], grad_fn=<AddmmBackward>)
tensor([[0.2646, 1.2385]], grad_fn=<AddmmBackward>)
tensor([[0.7558, 0.7941]], grad_fn=<AddmmBackward>)
tensor([[-0.0688,  1.4722]], grad_fn=<AddmmBackward>)
tensor([[-0.5056,  1.4102]], grad_fn=<AddmmBackward>)
tensor([[-1.1238,  3.5921]], grad_fn=<AddmmBackward>)
tensor([[ 0.7480, -0.6171]], grad_fn=<AddmmBackward>)
tensor([[-1.9815,  3.9353]], grad_fn=<AddmmBackward>)
tensor([[-0.4221,  2.7896]], grad_fn=<AddmmBackward>)
tensor([[-0.8572,  2.3309]], grad_fn=<AddmmBackward>)
tensor([[ 3.0234, -1.1528]], grad_fn=<AddmmBackward>)
tensor([[-1.0839,  3.9767]], grad_fn=<AddmmBackward>)
tensor([[ 0.7926, -0.5760]], grad_fn=<AddmmBackward>)
tensor([[-1.0330,  3.0932]], grad_fn=<AddmmBackward>)
tensor([[0.2398, 0.4694]], grad_fn=<AddmmBackward>)
tensor([[ 1.4427, -0.0935]], grad_fn

In [2]:
# Make the data frame withthe soft probabilities, predicted class labels and actual labels
df['school_id'] = ids
df['label'] = label
df['pass'] = cpass
df['fail'] = cfail
df['pred'] = 99
df['correct'] = 0

df['pred'][df['pass'] >= .50] = 0
df['pred'][df['pass'] < .50] = 1

df["correct"][(df['label'] == 0) & (df["pred"] == 0)] = 1
df["correct"][(df['label'] == 1) & (df["pred"] == 1)] = 1

print(df.head())
print(df['correct'].value_counts())

  school_id  label      pass      fail  pred  correct
0    300897      0  0.819868  0.180132     0        1
1    107371      0  0.152147  0.847853     1        0
2    305290      1  0.115126  0.884874     1        1
3    105374      0  0.997232  0.002768     0        1
4    106480      0  0.725892  0.274108     0        1
1    80
0    20
Name: correct, dtype: int64


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/loca

In [3]:
80 / 100

0.8

In [4]:
df.to_csv("./GoogleStaticPreds.csv", index = False)